In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import sqlite3
sqlite3.register_adapter(np.int64, int)
import re

db = sqlite3.connect('../stock.db' , isolation_level=None)

# 不縮寫成科學記號
pd.set_option('display.float_format',lambda x : '%.2f' % x)

# 展開所有dataframe columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def getYield(year, TYPEK='sii'):
    if year >= 1000:
        year -= 1911
        
    url = 'https://mops.twse.com.tw/server-java/t05st09sub'

    r = requests.post(url, {
        'step':1,
        'TYPEK':TYPEK,  # sii上市, otc上櫃
        'YEAR':str(year),
        'qryType': 1
    })
    
    r.encoding = 'big5'
    dfs = pd.read_html(StringIO(r.text), header=None)
    
    df = pd.concat(dfs[3:], axis=0, sort=False)  # 2021 -> 3 , 2022 -> 2
    
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
        
    return df


In [3]:
YEAR = 2023
df_sii = getYield(YEAR)
df_otc = getYield(YEAR, 'otc')

In [4]:
print(len(df_sii))
print(len(df_otc))
df = df_sii.append(df_otc, ignore_index=True)
print(len(df))
df

1010
818
1828


,公司代號 名稱,決議（擬議）進度,股利所屬 年(季)度,股利所屬 期間,期別,董事會決議\t（擬議）股\t利分派日,股東會 日期,期初未分配盈餘/待彌補虧損(元),本期淨利(淨損)(元),可分配 盈餘(元),分配後期末未分配盈餘(元),盈餘分配\t之現金股利\t(元/股),法定盈餘公積發放之現金(元/股),資本公積發放之現金(元/股),股東配發\t之現金(股利)總金額(元),盈餘轉\t增資配股\t(元/股),法定盈餘\t公積轉增資\t配股(元/股),資本公積\t轉增資配股\t(元/股),股東配股\t總股數(股),摘錄公司章程-股利分派部分,備註
0,1217 - 愛之味,董事會決議,111年年度,111/01/01~111/12/31,1,112/03/13,不適用,14338406,277889562,266533462,68728128,0.40,0.00,0.00,197805334,0.00,0.00,0.00,0,第三十條之一 公司年度總決算如有盈餘，除預提應繳稅捐，彌 補歷年虧損外，應先提撥百分之十為法...,無
1,1218 - 泰山,董事會決議,112年第1季,112/01/01~112/03/31,1,112/05/05,不適用,3620962604,-69949687,3551012917,3551012917,0.00,0.00,0.00,0,0.00,0.00,0.00,0,本公司目前產業發展成熟，獲利穩定，惟鑑於未來數年仍有重大之擴充產能及垂直發展之計劃，故盈餘之...,無
2,1218 - 泰山,董事會決議,111年第4季,111/10/01~111/12/31,1,112/05/05,不適用,663066273,5488771289,5620958756,3620962604,4.00,0.00,0.00,1999996152,0.00,0.00,0.00,0,本公司目前產業發展成熟，獲利穩定，惟鑑於未來數年仍有重大之擴充產能及垂直發展之計劃，故盈餘之...,111年全年現金股利分配為每股4元
3,1219 - 福壽,現金股利經董事會決議、增資配股經董事會擬議,111年下半年,111/07/01~111/12/31,1,112/04/18,112/06/09,546298282,-14703756,535150835,0,0.50,0.00,0.00,160824944,0.30,0.00,0.00,9649496,本公司盈餘分派或虧損撥補得於每半會計年度終了後為之。每半會計年度決算如有盈餘， 應先預估並保...,無
4,1220 - 台榮,董事會擬議,111年年度,111/01/01~111/12/31,1,112/03/23,112/06/21,11337247,45877602,53191596,17776008,0.20,0.00,0.00,35415588,0.00,0.00,0.00,0,無,無
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,9949 - 琉園,董事會擬議,111年年度,111/01/01~111/12/31,1,112/03/17,112/06/27,0,-35246863,-33710462,-33710462,0.00,0.00,0.00,0,0.00,0.00,0.00,0,第三十一條之一： 本公司年度決算如有盈餘，依法繳納稅捐，彌補累積虧損後，再提10%為法定盈餘...,"董事會決議 : 資本公積－股本溢價彌補虧損 13,673,269元 資本公積－庫藏股票交易彌..."
1824,9950 - 萬國通,董事會擬議,111年年度,111/01/01~111/12/31,1,112/03/24,112/06/21,-1201896700,-223512696,-1429826667,-1429826667,0.00,0.00,0.00,0,0.00,0.00,0.00,0,本公司年度總決算如有盈餘，應先提繳稅款，彌補以往年度虧損，次提百分十為法定盈餘公積，但法定盈...,無
1825,9951 - 皇田,董事會決議,111年年度,111/01/01~111/12/31,1,112/03/22,不適用,1502036682,411971753,1974010839,1726840839,3.30,0.00,0.00,247170000,0.00,0.00,0.00,0,本公司年度總決算如有盈餘，應先提繳營利事業所得稅，彌補以往年度虧損，如尚有餘額，應依法提列法...,無
1826,9960 - 邁達康,董事會擬議,111年年度,111/01/01~111/12/31,1,112/03/17,112/06/15,30317460,121692225,147373460,73469960,2.20,0.00,0.00,73903500,0.00,0.00,0.00,0,本公司每年度決算如有盈餘，依法繳納稅捐後，應先彌補歷年虧損，如尚有盈餘，先提列法定盈餘公積百...,無


In [5]:
def getId(r):
    try:
        return r['公司代號 名稱'].replace(" ", "").split("-")
    except:
        return [np.nan, np.nan]

df['id'], df['name'] = zip(*df.apply(getId, axis=1))

In [6]:
df['cash'] = df['盈餘分配\t之現金股利\t(元/股)'] + df['法定盈餘公積發放之現金(元/股)'] + df['資本公積發放之現金(元/股)']
df['stock'] = df['盈餘轉\t增資配股\t(元/股)'] + df['法定盈餘\t公積轉增資\t配股(元/股)'] + df['資本公積\t轉增資配股\t(元/股)']
df['total'] = df['cash'] + df['stock']

In [7]:
df[df['id']=='5483']

,公司代號 名稱,決議（擬議）進度,股利所屬 年(季)度,股利所屬 期間,期別,董事會決議\t（擬議）股\t利分派日,股東會 日期,期初未分配盈餘/待彌補虧損(元),本期淨利(淨損)(元),可分配 盈餘(元),分配後期末未分配盈餘(元),盈餘分配\t之現金股利\t(元/股),法定盈餘公積發放之現金(元/股),資本公積發放之現金(元/股),股東配發\t之現金(股利)總金額(元),盈餘轉\t增資配股\t(元/股),法定盈餘\t公積轉增資\t配股(元/股),資本公積\t轉增資配股\t(元/股),股東配股\t總股數(股),摘錄公司章程-股利分派部分,備註,id,name,cash,stock,total
1450,5483 - 中美晶,董事會決議,111年下半年,111/07/01~111/12/31,1,112/05/05,不適用,318460033,5629672917,6258923621,2858838046,5.80,0.00,0.00,3400085575,0.00,0.00,0.00,0,第二十七條：本公司年度如有獲利，應提撥百分之三至百分之十五為員工酬勞，由董事會決議以股票或現...,無,5483,中美晶,5.80,0.00,5.80


In [13]:
df[['id','name','cash','stock','total']].sort_values(by=['total'], ascending=False).head(100)

,id,name,cash,stock,total
358,2454,聯發科,76.00,0.00,76.00
423,2603,長榮,70.00,0.00,70.00
878,6669,緯穎,50.00,0.00,50.00
526,3008,大立光,46.00,0.00,46.00
1393,5274,信驊,45.00,0.00,45.00
836,6409,旭隼,43.50,0.00,43.50
633,3592,瑞鼎,41.00,0.00,41.00
549,3034,聯詠,37.00,0.00,37.00
1127,3293,鈊象,35.00,0.00,35.00
309,2379,瑞昱,27.00,0.00,27.00


# Write to DB

In [14]:
for index, row in df.iterrows(): 
    sql_insert = f'insert into yield (date, id, name, cash, stock, total) \
                   values (?,?,?,?,?,?)'
    db.execute(sql_insert, (YEAR, row['id'],row['name'],row['cash'],row['stock'],row['total']))


# Create Table

In [71]:
sql = 'create table if not exists yield \
           (date DATE, \
            id TEXT ,\
            name TEXT ,\
            cash INTEGER ,\
            stock INTEGER ,\
            total INTEGER);'

db.execute(sql)

In [15]:
# check in db
sql = f'select * from yield'
df_yield = pd.read_sql_query(sql, db)
df_yield

,date,id,name,cash,stock,total
0,2022,1101,台泥,1.00,1.00,2.00
1,2022,1101B,台泥乙特,1.75,0.00,1.75
2,2022,1102,亞泥,3.40,0.00,3.40
3,2022,1104,環泥,1.00,0.00,1.00
4,2022,1108,幸福,0.60,0.00,0.60
...,...,...,...,...,...,...
5478,2023,9949,琉園,0.00,0.00,0.00
5479,2023,9950,萬國通,0.00,0.00,0.00
5480,2023,9951,皇田,3.30,0.00,3.30
5481,2023,9960,邁達康,2.20,0.00,2.20


In [16]:
db.close()